In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# **# Week 4 — Data Cleaning & Dataset Integration**
This notebook performs structured cleaning on each dataset, standardizes schemas, fixes data types, removes inconsistencies, handles missing values, and merges all major sources into a unified `master_student_dataset`.

**Deliverables for Week 4:**
- Cleaned per-dataset CSV files
- Merged master dataset (`master_student_dataset.csv`)
- Documentation of cleaning steps inside this notebook


In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)


Define paths

In [3]:
# Mendeley dataset
path_mendeley = "/content/drive/My Drive/Practicum_Project_2/Data/RawData/Student_Dataset_Mendeley_data/student_dataset.csv"

# Mental health dataset
path_mental = "/content/drive/My Drive/Practicum_Project_2/Data/RawData/Student_Mental_Health_Resilience_Dataset/mental_health_dataset.csv"

# Kaggle Performance & Attendance – contains 5 files
path_attendance    = "/content/drive/My Drive/Practicum_Project_2/Data/RawData/Student_Performance_and_Attendance_Dataset/attendance.csv"
path_homework      = "/content/drive/My Drive/Practicum_Project_2/Data/RawData/Student_Performance_and_Attendance_Dataset/homework.csv"
path_performance   = "/content/drive/My Drive/Practicum_Project_2/Data/RawData/Student_Performance_and_Attendance_Dataset/performance.csv"
path_students      = "/content/drive/My Drive/Practicum_Project_2/Data/RawData/Student_Performance_and_Attendance_Dataset/students.csv"
path_teacherparent = "/content/drive/My Drive/Practicum_Project_2/Data/RawData/Student_Performance_and_Attendance_Dataset/teacher_parent_communication.csv"


Load datasets into DataFrames

In [4]:
# Load academic datasets
df_att        = pd.read_csv(path_attendance)
df_hw         = pd.read_csv(path_homework)
df_perf       = pd.read_csv(path_performance)
df_students   = pd.read_csv(path_students)
df_tp         = pd.read_csv(path_teacherparent)

# Load additional datasets
df_mendeley = pd.read_csv(path_mendeley)
df_mental   = pd.read_csv(path_mental)


Confirm loaded shapes

In [5]:
df_att.shape, df_hw.shape, df_perf.shape, df_students.shape, df_tp.shape, df_mendeley.shape, df_mental.shape


((364680, 4),
 (60780, 7),
 (36468, 5),
 (12156, 5),
 (24312, 4),
 (9000, 10),
 (500, 13))

Clean Each Dataset Before Merging                
Standardize Column Names -(Consistent naming makes merging easier and reduces errors)

In [6]:
def clean_columns(df, name="df"):
    df.columns = (
        df.columns.str.strip()
                  .str.lower()
                  .str.replace(' ', '_')
                  .str.replace('%', 'pct')
                  .str.replace('-', '_')
    )
    print(f"[✓] Cleaned column names for: {name}")
    return df

dfs_named = {
    "attendance": df_att,
    "homework": df_hw,
    "performance": df_perf,
    "students": df_students,
    "teacher_parent": df_tp,
    "mendeley": df_mendeley,
    "mental_health": df_mental
}

for name, d in dfs_named.items():
    clean_columns(d, name)


[✓] Cleaned column names for: attendance
[✓] Cleaned column names for: homework
[✓] Cleaned column names for: performance
[✓] Cleaned column names for: students
[✓] Cleaned column names for: teacher_parent
[✓] Cleaned column names for: mendeley
[✓] Cleaned column names for: mental_health


# **Convert Data Types (Dates, Numbers)**

Attendance Dates

In [7]:
df_att['date'] = pd.to_datetime(df_att['date'], errors='coerce')
print("[✓] Parsed attendance dates")


[✓] Parsed attendance dates


Homework due dates

In [8]:
df_hw['due_date'] = pd.to_datetime(df_hw['due_date'], errors='coerce')
print("[✓] Parsed homework due dates")

[✓] Parsed homework due dates


Performance: numeric fields

In [10]:
df_perf['exam_score'] = pd.to_numeric(df_perf['exam_score'], errors='coerce')
print("[✓] Converted exam_score to numeric")

df_perf['homework_completion_pct'] = (
    df_perf['homework_completion_pct']
          .astype(str)
          .str.replace('%', '')
)

df_perf['homework_completion_pct'] = pd.to_numeric(df_perf['homework_completion_pct'], errors='coerce')
print("[✓] Converted homework_completion_pct to numeric")


[✓] Converted exam_score to numeric
[✓] Converted homework_completion_pct to numeric


Students: DOB → Age

In [11]:
df_students['date_of_birth'] = pd.to_datetime(df_students['date_of_birth'], errors='coerce')
df_students['age'] = (pd.Timestamp.today() - df_students['date_of_birth']).dt.days // 365
print("[✓] Converted DOB to age")


[✓] Converted DOB to age


Mental health numeric fields

In [12]:
df_mental['sleep_hours'] = pd.to_numeric(df_mental['sleep_hours'], errors='coerce')
df_mental['sentiment_score'] = pd.to_numeric(df_mental['sentiment_score'], errors='coerce')
print("[✓] Converted mental health numeric fields")


[✓] Converted mental health numeric fields


# **Standardize Categorical Values**                         
Gender cleanup

In [14]:
print("\n--- Cleaning categorical columns ---")

# List of possible categorical columns you want to fix
cat_columns = ['gender', 'parent_education', 'school_type']

for col in cat_columns:
    if col in df_perf.columns:
        print(f"\nColumn found: '{col}' — cleaning...")

        print("Before:")
        print(df_perf[col].value_counts(dropna=False))

        # Convert to string, strip spaces, lowercase
        df_perf[col] = (
            df_perf[col]
            .astype(str)
            .str.strip()
            .str.lower()
        )

        # Special fix for gender only if gender exists
        if col == 'gender':
            df_perf[col] = df_perf[col].replace({
                'm': 'male',
                'f': 'female',
                'nan': 'unknown'
            })

        print("After:")
        print(df_perf[col].value_counts(dropna=False))

    else:
        print(f"Column missing: '{col}' — skipping...")



--- Cleaning categorical columns ---
Column missing: 'gender' — skipping...
Column missing: 'parent_education' — skipping...
Column missing: 'school_type' — skipping...


**Auto-detect categorical columns** - No need to manually list them.           
**Clean string formats** - Strip spaces, lowercase, normalize labels.         
**Fix special cases automatically**
gender → male/female/unknown
attendance_status → present/absent
homework status → normalized
emoji values become text
inconsistent labels removed

# **Handle Missing Values**

In [15]:
print("\n================ Handling Missing Values ================")

def handle_missing_values(df, name):
    print(f"\n\n--- Missing Value Treatment: {name} ---")

    # Show missing summary BEFORE
    print("\nBefore cleaning — Missing values per column:")
    print(df.isnull().sum().sort_values(ascending=False).head(15))

    # Numeric columns → fill with median
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns
    print("\nNumeric columns detected:")
    print(list(num_cols))

    for col in num_cols:
        missing_count = df[col].isnull().sum()
        if missing_count > 0:
            median_val = df[col].median()
            df[col].fillna(median_val, inplace=True)
            print(f"Filled {missing_count} missing values in '{col}' with median = {median_val}")

    # Categorical columns → fill with 'unknown'
    cat_cols = df.select_dtypes(include=['object']).columns
    print("\nCategorical columns detected:")
    print(list(cat_cols))

    for col in cat_cols:
        missing_count = df[col].isnull().sum()
        if missing_count > 0:
            df[col].fillna("unknown", inplace=True)
            print(f"Filled {missing_count} missing values in '{col}' with 'unknown'")

    # Show missing summary AFTER
    print("\nAfter cleaning — Missing values per column:")
    print(df.isnull().sum().sort_values(ascending=False).head(15))

    return df


# Apply to each dataset
df_perf = handle_missing_values(df_perf, "Performance")
df_att = handle_missing_values(df_att, "Attendance")
df_hw = handle_missing_values(df_hw, "Homework")
df_students = handle_missing_values(df_students, "Students")
df_mendeley = handle_missing_values(df_mendeley, "Mendeley Dataset")
df_mental = handle_missing_values(df_mental, "Mental Health")



================ Handling Missing Values ================


--- Missing Value Treatment: Performance ---

Before cleaning — Missing values per column:
student_id                 0
subject                    0
exam_score                 0
homework_completion_pct    0
teacher_comments           0
dtype: int64

Numeric columns detected:
['exam_score', 'homework_completion_pct']

Categorical columns detected:
['student_id', 'subject', 'teacher_comments']

After cleaning — Missing values per column:
student_id                 0
subject                    0
exam_score                 0
homework_completion_pct    0
teacher_comments           0
dtype: int64


--- Missing Value Treatment: Attendance ---

Before cleaning — Missing values per column:
student_id           0
date                 0
subject              0
attendance_status    0
dtype: int64

Numeric columns detected:
[]

Categorical columns detected:
['student_id', 'subject', 'attendance_status']

After cleaning — Missing values per

/tmp/ipython-input-515477003.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_val, inplace=True)
/tmp/ipython-input-515477003.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.

**Performance, Attendance, Mendeley, Mental Health** -z ero missing values → nothing to fill. This is expected because those datasets are well-structured.

**Students dataset** - Missing age fixed with the median age = 13,Missing emergency_contact fixed → "unknown",date_of_birth still missing (1205 rows)
You will fix DOB later in next Step

**Homework dataset** - due_date has 12,358 missing,This is expected because most homework datasets from Kaggle had missing due dates

We can fix it by : drop missing due dates, or fill using assignment batches or keep as it is.

# **Removing Duplicates & Impossible Values**

In [16]:
print("\n================ Remove Duplicates & Impossible Values ================\n")

def clean_duplicates_and_impossible(df, name, id_col="student_id"):
    print(f"\n--- Cleaning Dataset: {name} ---")

    # Before-shape summary
    print(f"Initial shape: {df.shape}")

    # 1. Duplicate Check
    if id_col in df.columns:
        dup_count = df.duplicated(subset=[id_col]).sum()
        print(f"Duplicate rows based on '{id_col}': {dup_count}")
        df = df.drop_duplicates(subset=[id_col])
        print(f"Shape after removing duplicates: {df.shape}")
    else:
        print(f"Column '{id_col}' missing — skipping duplicate removal")

    # 2. Impossible value checks (dataset specific)

    # Attendance %
    if "attendance_pct" in df.columns:
        invalid_att = df[(df['attendance_pct'] < 0) | (df['attendance_pct'] > 100)].shape[0]
        print(f"Invalid attendance_pct rows (<0 or >100): {invalid_att}")
        df = df[(df['attendance_pct'] >= 0) & (df['attendance_pct'] <= 100)]

    # Exam Score sanity check
    if "exam_score" in df.columns:
        invalid_exam = df[(df['exam_score'] < 0) | (df['exam_score'] > 100)].shape[0]
        print(f"Invalid exam_score rows (<0 or >100): {invalid_exam}")
        df = df[(df['exam_score'] >= 0) & (df['exam_score'] <= 100)]

    # GPA range check (0–4 scale common)
    if "gpa" in df.columns:
        invalid_gpa = df[(df['gpa'] < 0) | (df['gpa'] > 4.0)].shape[0]
        print(f"Invalid GPA rows (<0 or >4): {invalid_gpa}")
        df = df[(df['gpa'] >= 0) & (df['gpa'] <= 4.0)]

    # Sleep hours check (0–24)
    if "sleep_hours" in df.columns:
        invalid_sleep = df[(df['sleep_hours'] < 0) | (df['sleep_hours'] > 24)].shape[0]
        print(f"Invalid sleep_hours rows (<0 or >24): {invalid_sleep}")
        df = df[(df['sleep_hours'] >= 0) & (df['sleep_hours'] <= 24)]

    # 3. Negative age check
    if "age" in df.columns:
        invalid_age = df[df['age'] < 0].shape[0]
        print(f"Invalid age rows (<0): {invalid_age}")
        df = df[df['age'] >= 0]

    # Final shape
    print(f"Final shape after cleaning: {df.shape}")
    print("--- Done ---\n")

    return df


# APPLY TO ALL DATASETS

df_perf = clean_duplicates_and_impossible(df_perf, "Performance")
df_att = clean_duplicates_and_impossible(df_att, "Attendance")
df_hw = clean_duplicates_and_impossible(df_hw, "Homework")
df_students = clean_duplicates_and_impossible(df_students, "Students")
df_mendeley = clean_duplicates_and_impossible(df_mendeley, "Mendeley")
df_mental = clean_duplicates_and_impossible(df_mental, "Mental Health")



================ Remove Duplicates & Impossible Values ================


--- Cleaning Dataset: Performance ---
Initial shape: (36468, 5)
Duplicate rows based on 'student_id': 24888
Shape after removing duplicates: (11580, 5)
Invalid exam_score rows (<0 or >100): 1654
Final shape after cleaning: (9926, 5)
--- Done ---


--- Cleaning Dataset: Attendance ---
Initial shape: (364680, 4)
Duplicate rows based on 'student_id': 352524
Shape after removing duplicates: (12156, 4)
Final shape after cleaning: (12156, 4)
--- Done ---


--- Cleaning Dataset: Homework ---
Initial shape: (60780, 7)
Duplicate rows based on 'student_id': 48715
Shape after removing duplicates: (12065, 7)
Final shape after cleaning: (12065, 7)
--- Done ---


--- Cleaning Dataset: Students ---
Initial shape: (12156, 6)
Duplicate rows based on 'student_id': 0
Shape after removing duplicates: (12156, 6)
Invalid age rows (<0): 0
Final shape after cleaning: (12156, 6)
--- Done ---


--- Cleaning Dataset: Mendeley ---
Initial 

**Performance Dataset**
Started with 36,468 rows. After removing duplicates → 11,580 unique students. After removing impossible exam scores (>100 or <0) -> 9,926 valid rows

This is normal. Large performance tables often have multiple scores per student → duplicates. You now have a clean, valid academic performance table.

**Attendance Dataset**
Started with 364,680 rows. After removing duplicates → 12,156 unique students    
This aligns with the Students dataset (also 12,156).        
Perfect — merge-ready.

**Homework Dataset**
Started with 60,780. After removing duplicates → 12,065 unique students           
Good.This dataset is slightly smaller than attendance/students — normal, as some student IDs missing homework data.

**Students Dataset**
Clean, no duplicates, valid ranges. This will be your primary key table for merging.

**Mendeley Dataset**
No student_id → cannot merge directly. You will need to join this using some common fields (e.g., gender, age, parental education) OR keep it as a separate analysis dataset.

**Mental Health Dataset**
Fully clean. 500 rows only - This dataset may need sampling alignment before merging (otherwise merging will drop 90% of students).

### **Save Cleaned Datasets**

In [18]:
import os

save_path = "/content/drive/My Drive/Practicum_Project_2/Data/CleanData/"
os.makedirs(save_path, exist_ok=True)

datasets_to_save = {
    "attendance_cleaned.csv": df_att,
    "homework_cleaned.csv": df_hw,
    "performance_cleaned.csv": df_perf,
    "students_cleaned.csv": df_students,
    "mental_health_cleaned.csv": df_mental,
    "mendeley_raw_cleaned.csv": df_mendeley  # <- fixed name
}

print("=== Saving Cleaned Datasets ===")
for filename, df in datasets_to_save.items():
    full_path = os.path.join(save_path, filename)
    df.to_csv(full_path, index=False)
    print(f"Saved: {filename}  | Shape: {df.shape}")

print("=== All cleaned datasets saved successfully! ===")


=== Saving Cleaned Datasets ===
Saved: attendance_cleaned.csv  | Shape: (12156, 4)
Saved: homework_cleaned.csv  | Shape: (12065, 7)
Saved: performance_cleaned.csv  | Shape: (9926, 5)
Saved: students_cleaned.csv  | Shape: (12156, 6)
Saved: mental_health_cleaned.csv  | Shape: (500, 13)
Saved: mendeley_raw_cleaned.csv  | Shape: (9000, 10)
=== All cleaned datasets saved successfully! ===


**Merge by aggregating first (safer & smaller master)**

In [20]:
print("=== Converting student_id columns to string ===")

dfs = {
    "students": df_students,
    "attendance": df_att,
    "homework": df_hw,
    "performance": df_perf,
    "mental": df_mental,
    "messages": df_tp
}

for name, d in dfs.items():
    if "student_id" in d.columns:
        d["student_id"] = d["student_id"].astype(str).str.strip()
        print(f"{name}: converted student_id → string")

print("=== Conversion complete ===")


=== Converting student_id columns to string ===
students: converted student_id → string
attendance: converted student_id → string
homework: converted student_id → string
performance: converted student_id → string
mental: converted student_id → string
messages: converted student_id → string
=== Conversion complete ===


/tmp/ipython-input-3221307313.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d["student_id"] = d["student_id"].astype(str).str.strip()
/tmp/ipython-input-3221307313.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d["student_id"] = d["student_id"].astype(str).str.strip()


In [24]:
import os
import pandas as pd

# ================= Output Directory Exists =================
out_dir = "/content/drive/My Drive/Practicum_Project_2/Data/Final"
os.makedirs(out_dir, exist_ok=True)
print(f"Output directory ready: {out_dir}\n")

# ================= Ensure student_id types are consistent =================
# Convert student_id to string for all datasets
dfs = [df_students, df_att, df_hw, df_perf, df_mental]
for df in dfs:
    if 'student_id' in df.columns:
        df['student_id'] = df['student_id'].astype(str)
print("All student_id columns converted to string.\n")

# ================= Create message summary safely =================
try:
    df_messages
except NameError:
    print("df_messages not found, creating empty summary.")
    df_messages = pd.DataFrame(columns=["student_id", "msg_count", "last_contact"])
else:
    # If df_messages exists, summarize
    df_messages['student_id'] = df_messages['student_id'].astype(str)
    if 'date' in df_messages.columns:
        df_messages = df_messages.groupby('student_id').agg(
            msg_count=('student_id', 'count'),
            last_contact=('date', 'max')
        ).reset_index()
    else:
        df_messages = df_messages.groupby('student_id').agg(
            msg_count=('student_id', 'count')
        ).reset_index()
print("Message dataset ready:\n", df_messages.head(), "\n")

# ================= Summarize / merge smaller datasets =================
# Attendance summary
df_att['present_flag'] = df_att['attendance_status'].astype(str).str.strip().str.lower().eq('present').astype(int)
att_summary = df_att.groupby('student_id').agg(attendance_rate=('present_flag','mean')).reset_index()
print("Attendance summary shape:", att_summary.shape)

# Homework summary
df_hw['done_flag'] = df_hw['status'].astype(str).str.strip().str.lower().isin(['done','✅','✔','complete','completed']).astype(int)
hw_summary = df_hw.groupby('student_id').agg(hw_done_rate=('done_flag','mean')).reset_index()
print("Homework summary shape:", hw_summary.shape)

# Performance summary
perf_summary = df_perf.groupby('student_id').agg(avg_score=('exam_score','mean')).reset_index()
print("Performance summary shape:", perf_summary.shape)

# ================= Merge everything =================
master = df_students.copy()
print("Base students shape:", master.shape)

master = master.merge(att_summary, on='student_id', how='left')
print("After attendance merge:", master.shape, "| attendance_rate missing:", master['attendance_rate'].isna().sum())

master = master.merge(hw_summary, on='student_id', how='left')
print("After homework merge:", master.shape, "| hw_done_rate missing:", master['hw_done_rate'].isna().sum())

master = master.merge(perf_summary, on='student_id', how='left')
print("After performance merge:", master.shape, "| avg_score missing:", master['avg_score'].isna().sum())

master = master.merge(df_mental[['student_id','stress_level','anxiety_score','depression_score','sentiment_score','sleep_hours']],
                      on='student_id', how='left')
print("After mental health merge:", master.shape, "| stress_level missing:", master['stress_level'].isna().sum())

master = master.merge(df_messages, on='student_id', how='left')
print("After message merge:", master.shape, "| msg_count missing:", master['msg_count'].isna().sum())

print("\nTop-level missing value fractions (first 10):\n", master.isna().mean().head(10))

# ================= Save master CSV =================
out_master = os.path.join(out_dir, "student_master.csv")
master.to_csv(out_master, index=False)
print(f"\nMaster dataset saved successfully: {out_master}")


Output directory ready: /content/drive/My Drive/Practicum_Project_2/Data/Final

All student_id columns converted to string.

df_messages not found, creating empty summary.
Message dataset ready:
 Empty DataFrame
Columns: [student_id, msg_count, last_contact]
Index: [] 

Attendance summary shape: (12156, 2)
Homework summary shape: (12065, 2)
Performance summary shape: (9926, 2)
Base students shape: (12156, 6)
After attendance merge: (12156, 7) | attendance_rate missing: 0
After homework merge: (12156, 8) | hw_done_rate missing: 91
After performance merge: (12156, 9) | avg_score missing: 2230
After mental health merge: (12156, 14) | stress_level missing: 12156
After message merge: (12156, 16) | msg_count missing: 12156

Top-level missing value fractions (first 10):
 student_id           0.000000
full_name            0.000000
date_of_birth        0.099128
grade_level          0.000000
emergency_contact    0.000000
age                  0.000000
attendance_rate      0.000000
hw_done_rate   

/tmp/ipython-input-3329585403.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['student_id'] = df['student_id'].astype(str)
/tmp/ipython-input-3329585403.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['student_id'] = df['student_id'].astype(str)
/tmp/ipython-input-3329585403.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/


Master dataset saved successfully: /content/drive/My Drive/Practicum_Project_2/Data/Final/student_master.csv


**Observations from the merge output:**

Attendance merged fully, no missing values.

Homework missing rate is low (~0.7%).

Performance missing ~18% (avg_score) — some students may not have exams recorded.

Mental health data is entirely missing for most students (expected if the dataset only has 500 entries).

Message counts are missing entirely (empty df_messages), so you may fill later if data is available.

The warnings (SettingWithCopyWarning) are not breaking the workflow — they are just reminders that we are modifying slices of DataFrames. You can ignore them for now

**Week 4 Summary Report**

In [25]:
# ==================== WEEK 4 SUMMARY REPORT ====================

import pandas as pd

# Load master dataset
master_path = "/content/drive/My Drive/Practicum_Project_2/Data/Final/student_master.csv"
master = pd.read_csv(master_path)

# Basic info
print("========== MASTER DATASET INFO ==========")
print(f"Master shape: {master.shape}")
print("\nColumns and missing values:")
missing_frac = master.isna().mean()
print(missing_frac.sort_values(ascending=False).head(10))

# Coverage per original dataset
print("\n========== DATASET COVERAGE ==========")
datasets = {
    "Attendance": "attendance_rate",
    "Homework": "hw_done_rate",
    "Performance": "avg_score",
    "Mental Health": "stress_level",
    "Messages": "msg_count"
}

for name, col in datasets.items():
    if col in master.columns:
        total = len(master)
        missing = master[col].isna().sum()
        covered = total - missing
        coverage_pct = covered / total * 100
        print(f"{name}: {covered}/{total} ({coverage_pct:.1f}%) covered, missing: {missing} ({100-coverage_pct:.1f}%)")
    else:
        print(f"{name}: column '{col}' not found")

# Optional: describe numeric columns
print("\n========== NUMERIC COLUMN SUMMARY ==========")
numeric_cols = master.select_dtypes(include='number').columns
print(master[numeric_cols].describe())

# Optional: quick check for duplicates
duplicates = master.duplicated(subset=['student_id']).sum()
print(f"\nDuplicate student_id rows in master: {duplicates}")


========== MASTER DATASET INFO ==========
Master shape: (12156, 16)

Columns and missing values:
sentiment_score     1.000000
last_contact        1.000000
sleep_hours         1.000000
msg_count           1.000000
anxiety_score       1.000000
depression_score    1.000000
stress_level        1.000000
avg_score           0.183449
date_of_birth       0.099128
hw_done_rate        0.007486
dtype: float64

========== DATASET COVERAGE ==========
Attendance: 12156/12156 (100.0%) covered, missing: 0 (0.0%)
Homework: 12065/12156 (99.3%) covered, missing: 91 (0.7%)
Performance: 9926/12156 (81.7%) covered, missing: 2230 (18.3%)
Mental Health: 0/12156 (0.0%) covered, missing: 12156 (100.0%)
Messages: 0/12156 (0.0%) covered, missing: 12156 (100.0%)

========== NUMERIC COLUMN SUMMARY ==========
                age  attendance_rate  hw_done_rate    avg_score  stress_level  \
count  12156.000000     12156.000000  12065.000000  9926.000000           0.0   
mean      12.755100         0.249424      0.4977

**Shape**

12156 students, 16 columns — consistent with your base students table.

**Missing Value Summary**

100% missing for mental health columns (stress_level, anxiety_score, depression_score, sentiment_score, sleep_hours)
→ The mental health dataset didn’t merge. Likely student_id type mismatch or missing IDs.

100% missing for message columns (msg_count, last_contact)
→ Messages dataset was empty. Safe to keep as placeholder if you plan to add later.

Homework & Performance have some missing values (≈0.7% and 18.3% respectively)

**Numeric Summary**

attendance_rate mean ~25%, median 0 → many students never marked present

hw_done_rate median 0.5 → roughly half homework completed

avg_score median 70 → reasonable academic performance distribution

Mental health and messages columns are all NaN → need fixing

**Duplicates**

No duplicate student_id rows → good

In [26]:
print(master['student_id'].dtype, df_mental['student_id'].dtype)


object object


In [27]:
master['student_id'] = master['student_id'].astype(str)
df_mental['student_id'] = df_mental['student_id'].astype(str)
master = master.merge(df_mental[['student_id','stress_level','anxiety_score','depression_score','sentiment_score','sleep_hours']], on='student_id', how='left')


In [28]:
master['avg_score'].fillna(master['avg_score'].median(), inplace=True)
master['hw_done_rate'].fillna(0, inplace=True)


/tmp/ipython-input-934891078.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  master['avg_score'].fillna(master['avg_score'].median(), inplace=True)
/tmp/ipython-input-934891078.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

In [29]:
master.to_csv("/content/drive/My Drive/Practicum_Project_2/Data/Final/student_master_updated.csv", index=False)


File is saved in new updated file student_master_updated.csv under final document